In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.dpi'] = 70 #display 70 dpi in Jupyter Notebook, may consider100 dpi 
plt.rcParams['savefig.dpi'] = 300 #define 300 dpi for saving figures

import seaborn as sns
## here are some settings 
sns.set_style('whitegrid')
sns.set(rc={"figure.dpi":70, 'savefig.dpi':300}) #defining dpi setting
sns.set_context('notebook')
sns.set_style("ticks")
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('retina')
# Tells matplotlib to display images inline instead of a new window
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
random.seed(1000)

from time import time
import timeit #imports timeit module

C:\Users\LMS-Khatrib\AppData\Local\Temp\ipykernel_8340\641145125.py:13: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  set_matplotlib_formats('retina')


In [2]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.preprocessing import PolynomialFeatures

import sklearn.linear_model as skl_lm
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn import neighbors
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis 
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.tree import DecisionTreeRegressor 
from sklearn.tree import DecisionTreeClassifier 
from sklearn.tree import export_graphviz
from sklearn.svm import SVC

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import train_test_split

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import mean_squared_error

from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [3]:
df2017 = pd.read_csv('JORN21_dataset_v1.csv', na_values = '?').dropna()
df2021 = pd.read_csv("SRER21_dataset_v1.csv", na_values = '?').dropna()

df2017 = df2017.reindex(columns=["OID_", "Id", "gridcode","Shape_Length", "Shape_Area","CH_mean", "ARVI_mean","ARVI_max","ARVI_med","EVI_mean","EVI_max","EVI_med","NDVI_mean","NDVI_max","NDVI_med","SAVI_mean","SAVI_max","SAVI_med", "Veg_class"])
df2021 = df2021.reindex(columns=["OID_", "Id", "gridcode","Shape_Length", "Shape_Area","CH_mean", "ARVI_mean","ARVI_max","ARVI_med","EVI_mean","EVI_max","EVI_med","NDVI_mean","NDVI_max","NDVI_med","SAVI_mean","SAVI_max","SAVI_med", "Veg_class"])

bigtest1_df = pd.read_csv('SRER21_pred.csv', na_values='?')
bg1 = bigtest1_df.drop(columns=["Veg_class"])

dffull2021 = bg1.dropna()
dffull2021 = dffull2021.reindex(columns = ["OID_", "Id", "gridcode","Shape_Length", "Shape_Area","CH_mean", "ARVI_mean","ARVI_max","ARVI_med","EVI_mean","EVI_max","EVI_med","NDVI_mean","NDVI_max","NDVI_med","SAVI_mean","SAVI_max","SAVI_med", "Veg_class"]).fillna(0)

In [4]:
df2017.Veg_class = df2017.Veg_class.map({'non-woody':0, 'woody':1})
df2021.Veg_class = df2021.Veg_class.map({'non-woody':0, 'woody':1})

# Predicting models (JORN 2021 -> SRER 2021)

In [5]:
crossvalidation = KFold(n_splits=5, random_state=2, shuffle=True)
model = DecisionTreeClassifier(max_depth=6)

#criterion='entropy'
#scores = cross_val_score(model, X, y, cv=crossvalidation)
#print('Accuracy of each fold: \n {}'.format(scores))
#print()
#print("Avg accuracy: {}".format(scores.mean()))

acc_score = [];
Truth = [];
Output = [];

start_time = 0
for train_index, test_index in crossvalidation.split(df2017): 
    
    X_train = df2017.iloc[train_index, 5:18]
    X_test = df2017.iloc[test_index, 5:18]
    Y_train = df2017.iloc[train_index, -1]
    Y_test = df2017.iloc[test_index, -1]

    model.fit(X_train, Y_train)
    pred_values = model.predict(X_test)

    acc = accuracy_score(Y_test, pred_values)
    acc_score.append(acc)

    Truth.extend(Y_test.values.reshape(Y_test.shape[0])) 
    Output.extend(pred_values)

elapsed = timeit.default_timer() - start_time



print("---Run time is %s seconds ---" % elapsed)
print()
print('Accuracy of each fold: \n {}'.format(acc_score))
print("Avg accuracy: {}".format(np.mean(acc_score)))
print('Std of accuracy : \n{}'.format(np.std(acc_score)))
print()
print(confusion_matrix(Truth, Output))
print()
print(classification_report(Truth, Output))

cm = confusion_matrix(Truth, Output)

sensitivity = cm[0][0]/(cm[0][0]+cm[0][1])
specificity = cm[1][1]/(cm[1][0]+cm[1][1])
precision = (cm[0][0])/(cm[0][0]+cm[1][0])
f1_score = (2*precision*sensitivity)/(precision+sensitivity)


print("Sensitivity: {}".format(specificity))
print("Specificity: {}".format(sensitivity))
print("precision: {}".format(precision))
print("f1_score: {}".format(f1_score))


---Run time is 7.8061879 seconds ---

Accuracy of each fold: 
 [0.8298368298368298, 0.8317757009345794, 0.794392523364486, 0.8294392523364486, 0.794392523364486]
Avg accuracy: 0.815967365967366
Std of accuracy : 
0.01763351848884137

[[ 498  200]
 [ 194 1249]]

              precision    recall  f1-score   support

           0       0.72      0.71      0.72       698
           1       0.86      0.87      0.86      1443

    accuracy                           0.82      2141
   macro avg       0.79      0.79      0.79      2141
weighted avg       0.82      0.82      0.82      2141

Sensitivity: 0.8655578655578655
Specificity: 0.7134670487106017
precision: 0.7196531791907514
f1_score: 0.7165467625899281


In [6]:
model = DecisionTreeClassifier(max_depth=6)
acc_score = [];
Truth = [];
Output = [];
testing = df2021.iloc[:,-1]
model.fit(df2017.iloc[:, 5:18], df2017.iloc[:,-1])
pred_values = model.predict(df2021.iloc[:, 5:18])

acc = accuracy_score(testing, pred_values)
acc_score.append(acc)
Truth.extend(testing.values.reshape(testing.shape[0])) 
Output.extend(pred_values)

cm = confusion_matrix(Truth, Output)

sensitivity = cm[0][0]/(cm[0][0]+cm[0][1])
specificity = cm[1][1]/(cm[1][0]+cm[1][1])
precision = (cm[0][0])/(cm[0][0]+cm[1][0])
f1_score = (2*precision*sensitivity)/(precision+sensitivity)


print("Sensitivity: {}".format(specificity))
print("Specificity: {}".format(sensitivity))
print("precision: {}".format(precision))
print("f1_score: {}".format(f1_score))
print(acc_score)

Sensitivity: 0.953224043715847
Specificity: 0.22331838565022422
precision: 0.699438202247191
f1_score: 0.33854520734194427
[0.7140337986774431]


In [7]:
random.seed(1000)
bigtest_df = dffull2021
finalPredicted = model.predict(bigtest_df.iloc[:,5:18])
bigtest_df["Veg_class"] = finalPredicted

In [8]:
woodyarea = 0
nonwoodyarea = 0 
totalarea = 0
index = -1
woody = 0
nw = 0
#print(bigtest_df.iat[index,4])
area = bigtest_df["Shape_Area"]
#print(area)

for i in bigtest_df["Veg_class"]:
    index += 1
    if i == 1:
        totalarea += bigtest_df.iat[index,4]
        woodyarea += bigtest_df.iat[index,4]
        #print(woodyarea)
        woody += 1
    if i == 0:
        totalarea += bigtest_df.iat[index,4]
        nonwoodyarea += bigtest_df.iat[index,4]
        #print(nonwoodyarea)
        nw += 1

FWCdt = woodyarea / totalarea * 100

In [9]:
#Bagging Model 

crossvalidation = KFold(n_splits=5, random_state=2, shuffle=True)
model = RandomForestClassifier(max_features = 10, random_state = 2)
#scores = cross_val_score(model, X, y, cv=crossvalidation)
#print('Accuracy of each fold: \n {}'.format(scores))
#print()
#print("Avg accuracy: {}".format(scores.mean()))
#print(model)


acc_score = [];
Truth = [];
Output = [];

for train_index, test_index in crossvalidation.split(df2017): 
    
    X_train = df2017.iloc[train_index, 5:18]
    X_test = df2017.iloc[test_index, 5:18]
    Y_train = df2017.iloc[train_index, -1]
    Y_test = df2017.iloc[test_index, -1]

    model.fit(X_train, Y_train)
    pred_values = model.predict(X_test)

    acc = accuracy_score(Y_test, pred_values)
    acc_score.append(acc)

    Truth.extend(Y_test.values.reshape(Y_test.shape[0])) 
    Output.extend(pred_values)

elapsed = timeit.default_timer() - start_time



print("---Run time is %s seconds ---" % elapsed)
print()
print('Accuracy of each fold: \n {}'.format(acc_score))
print("Avg accuracy: {}".format(np.mean(acc_score)))
print('Std of accuracy : \n{}'.format(np.std(acc_score)))
print()
print(confusion_matrix(Truth, Output))
print()
print(classification_report(Truth, Output))

cm = confusion_matrix(Truth, Output)

sensitivity = cm[0][0]/(cm[0][0]+cm[0][1])
specificity = cm[1][1]/(cm[1][0]+cm[1][1])
precision = (cm[0][0])/(cm[0][0]+cm[1][0])
f1_score = (2*precision*sensitivity)/(precision+sensitivity)

print("Sensitivity: {}".format(specificity))
print("Specificity: {}".format(sensitivity))
print("precision: {}".format(precision))
print("f1_score: {}".format(f1_score))

#statallfeatures_dic['Run Time'].append(elapsed)
#statallfeatures_dic['Accuracy'].append(np.mean(acc_score))
#statallfeatures_dic['Standard Error'].append(np.std(acc_score))
#statallfeatures_dic['Sensitivity'].append(sensitivity)
#statallfeatures_dic['Specificity'].append(specificity)
#statallfeatures_dic['Precision'].append(precision)
#statallfeatures_dic['F1_Score'].append(f1_score)

#indaccs_dic['LogReg']=acc_score

---Run time is 52.17706 seconds ---

Accuracy of each fold: 
 [0.8344988344988346, 0.8387850467289719, 0.8411214953271028, 0.8247663551401869, 0.8341121495327103]
Avg accuracy: 0.8346567762455612
Std of accuracy : 
0.005603163454349531

[[ 496  202]
 [ 152 1291]]

              precision    recall  f1-score   support

           0       0.77      0.71      0.74       698
           1       0.86      0.89      0.88      1443

    accuracy                           0.83      2141
   macro avg       0.82      0.80      0.81      2141
weighted avg       0.83      0.83      0.83      2141

Sensitivity: 0.8946638946638946
Specificity: 0.7106017191977078
precision: 0.7654320987654321
f1_score: 0.736998514115899


In [10]:
model = RandomForestClassifier(max_features = 10, random_state = 2)
acc_score = [];
Truth = [];
Output = [];
testing = df2021.iloc[:,-1]
model.fit(df2017.iloc[:, 5:18], df2017.iloc[:,-1])
pred_values = model.predict(df2021.iloc[:, 5:18])

acc = accuracy_score(testing, pred_values)
acc_score.append(acc)
Truth.extend(testing.values.reshape(testing.shape[0])) 
Output.extend(pred_values)

cm = confusion_matrix(Truth, Output)

sensitivity = cm[0][0]/(cm[0][0]+cm[0][1])
specificity = cm[1][1]/(cm[1][0]+cm[1][1])
precision = (cm[0][0])/(cm[0][0]+cm[1][0])
f1_score = (2*precision*sensitivity)/(precision+sensitivity)


print("Sensitivity: {}".format(specificity))
print("Specificity: {}".format(sensitivity))
print("precision: {}".format(precision))
print("f1_score: {}".format(f1_score))
print(acc_score)

Sensitivity: 0.9628415300546448
Specificity: 0.1681614349775785
precision: 0.6880733944954128
f1_score: 0.2702702702702703
[0.7024246877296105]


In [11]:
random.seed(1000)
bigtest_df = dffull2021
finalPredicted = model.predict(bigtest_df.iloc[:,5:18])
bigtest_df["Veg_class"] = finalPredicted

In [12]:
woodyarea = 0
nonwoodyarea = 0 
totalarea = 0
index = -1
woody = 0
nw = 0
#print(bigtest_df.iat[index,4])
area = bigtest_df["Shape_Area"]
#print(area)

for i in bigtest_df["Veg_class"]:
    index += 1
    if i == 1:
        totalarea += bigtest_df.iat[index,4]
        woodyarea += bigtest_df.iat[index,4]
        #print(woodyarea)
        woody += 1
    if i == 0:
        totalarea += bigtest_df.iat[index,4]
        nonwoodyarea += bigtest_df.iat[index,4]
        #print(nonwoodyarea)
        nw += 1

FWCbag = woodyarea / totalarea * 100

In [13]:
crossvalidation = KFold(n_splits=5, random_state=2, shuffle=True)
model = AdaBoostClassifier(n_estimators=500, learning_rate = 0.1, algorithm="SAMME.R", random_state=2)
#scores = cross_val_score(model, X, y, cv=crossvalidation)
#print('Accuracy of each fold: \n {}'.format(scores))
#print()
#print("Avg accuracy: {}".format(scores.mean()))
#scores = cross_val_score(model, X, y, cv=crossvalidation)
#print('Accuracy of each fold: \n {}'.format(scores))
#print()
#print("Avg accuracy: {}".format(scores.mean()))
#print(model)


acc_score = [];
Truth = [];
Output = [];

for train_index, test_index in crossvalidation.split(df2017): 
    
    X_train = df2017.iloc[train_index, 5:18]
    X_test = df2021.iloc[test_index, 5:18]
    Y_train = df2017.iloc[train_index, -1]
    Y_test = df2021.iloc[test_index, -1]

    model.fit(X_train, Y_train)
    pred_values = model.predict(X_test)

    acc = accuracy_score(Y_test, pred_values)
    acc_score.append(acc)

    Truth.extend(Y_test.values.reshape(Y_test.shape[0])) 
    Output.extend(pred_values)

elapsed = timeit.default_timer() - start_time



print("---Run time is %s seconds ---" % elapsed)
print()
print('Accuracy of each fold: \n {}'.format(acc_score))
print("Avg accuracy: {}".format(np.mean(acc_score)))
print('Std of accuracy : \n{}'.format(np.std(acc_score)))
print()
print(confusion_matrix(Truth, Output))
print()
print(classification_report(Truth, Output))

cm = confusion_matrix(Truth, Output)

sensitivity = cm[0][0]/(cm[0][0]+cm[0][1])
specificity = cm[1][1]/(cm[1][0]+cm[1][1])
precision = (cm[0][0])/(cm[0][0]+cm[1][0])
f1_score = (2*precision*sensitivity)/(precision+sensitivity)

print("Sensitivity: {}".format(specificity))
print("Specificity: {}".format(sensitivity))
print("precision: {}".format(precision))
print("f1_score: {}".format(f1_score))

#statallfeatures_dic['Run Time'].append(elapsed)
#statallfeatures_dic['Accuracy'].append(np.mean(acc_score))
#statallfeatures_dic['Standard Error'].append(np.std(acc_score))
#statallfeatures_dic['Sensitivity'].append(sensitivity)
#statallfeatures_dic['Specificity'].append(specificity)
#statallfeatures_dic['Precision'].append(precision)
#statallfeatures_dic['F1_Score'].append(f1_score)

#indaccs_dic['LogReg']=acc_score

---Run time is 110.7476753 seconds ---

Accuracy of each fold: 
 [0.717948717948718, 0.7102803738317757, 0.7079439252336449, 0.7079439252336449, 0.6915887850467289]
Avg accuracy: 0.7071411454589025
Std of accuracy : 
0.008600269798364259

[[ 173  569]
 [  58 1341]]

              precision    recall  f1-score   support

           0       0.75      0.23      0.36       742
           1       0.70      0.96      0.81      1399

    accuracy                           0.71      2141
   macro avg       0.73      0.60      0.58      2141
weighted avg       0.72      0.71      0.65      2141

Sensitivity: 0.958541815582559
Specificity: 0.23315363881401618
precision: 0.7489177489177489
f1_score: 0.355601233299075


In [14]:
model = AdaBoostClassifier(n_estimators=500, learning_rate = 0.1, algorithm="SAMME.R", random_state=2)
acc_score = [];
Truth = [];
Output = [];
testing = df2021.iloc[:,-1]
model.fit(df2017.iloc[:, 5:18], df2017.iloc[:,-1])
pred_values = model.predict(df2021.iloc[:, 5:18])

acc = accuracy_score(testing, pred_values)
acc_score.append(acc)
Truth.extend(testing.values.reshape(testing.shape[0])) 
Output.extend(pred_values)

cm = confusion_matrix(Truth, Output)

sensitivity = cm[0][0]/(cm[0][0]+cm[0][1])
specificity = cm[1][1]/(cm[1][0]+cm[1][1])
precision = (cm[0][0])/(cm[0][0]+cm[1][0])
f1_score = (2*precision*sensitivity)/(precision+sensitivity)


print("Sensitivity: {}".format(specificity))
print("Specificity: {}".format(sensitivity))
print("precision: {}".format(precision))
print("f1_score: {}".format(f1_score))
print(acc_score)

Sensitivity: 0.9753005464480874
Specificity: 0.1780269058295964
precision: 0.7784313725490196
f1_score: 0.28978102189781024
[0.7140337986774431]


In [15]:
random.seed(1000)
bigtest_df = dffull2021
finalPredicted = model.predict(bigtest_df.iloc[:,5:18])
bigtest_df["Veg_class"] = finalPredicted

In [16]:
woodyarea = 0
nonwoodyarea = 0 
totalarea = 0
index = -1
woody = 0
nw = 0
#print(bigtest_df.iat[index,4])
area = bigtest_df["Shape_Area"]
#print(area)

for i in bigtest_df["Veg_class"]:
    index += 1
    if i == 1:
        totalarea += bigtest_df.iat[index,4]
        woodyarea += bigtest_df.iat[index,4]
        #print(woodyarea)
        woody += 1
    if i == 0:
        totalarea += bigtest_df.iat[index,4]
        nonwoodyarea += bigtest_df.iat[index,4]
        #print(nonwoodyarea)
        nw += 1

FWCada = woodyarea / totalarea * 100

In [17]:
crossvalidation = KFold(n_splits=5, random_state=2, shuffle=True)
model =  GradientBoostingClassifier(n_estimators = 500, 
                                           learning_rate = 0.1, 
                                           max_depth = 4, 
                                           random_state = 2)
#scores = cross_val_score(model, X, y, cv=crossvalidation)
#print('Accuracy of each fold: \n {}'.format(scores))
#print()
#print("Avg accuracy: {}".format(scores.mean()))

#scores = cross_val_score(model, X, y, cv=crossvalidation)
#print('Accuracy of each fold: \n {}'.format(scores))
#print()
#print("Avg accuracy: {}".format(scores.mean()))
#print(model)


acc_score = [];
Truth = [];
Output = [];

for train_index, test_index in crossvalidation.split(df2017): 
    
    X_train = df2017.iloc[train_index, 5:18]
    X_test = df2017.iloc[test_index, 5:18]
    Y_train = df2017.iloc[train_index, -1]
    Y_test = df2017.iloc[test_index, -1]

    model.fit(X_train, Y_train)
    pred_values = model.predict(X_test)

    acc = accuracy_score(Y_test, pred_values)
    acc_score.append(acc)

    Truth.extend(Y_test.values.reshape(Y_test.shape[0])) 
    Output.extend(pred_values)

elapsed = timeit.default_timer() - start_time



print("---Run time is %s seconds ---" % elapsed)
print()
print('Accuracy of each fold: \n {}'.format(acc_score))
print("Avg accuracy: {}".format(np.mean(acc_score)))
print('Std of accuracy : \n{}'.format(np.std(acc_score)))
print()
print(confusion_matrix(Truth, Output))
print()
print(classification_report(Truth, Output))

cm = confusion_matrix(Truth, Output)

sensitivity = cm[0][0]/(cm[0][0]+cm[0][1])
specificity = cm[1][1]/(cm[1][0]+cm[1][1])
precision = (cm[0][0])/(cm[0][0]+cm[1][0])
f1_score = (2*precision*sensitivity)/(precision+sensitivity)

print("Sensitivity: {}".format(specificity))
print("Specificity: {}".format(sensitivity))
print("precision: {}".format(precision))
print("f1_score: {}".format(f1_score))

#statallfeatures_dic['Run Time'].append(elapsed)
#statallfeatures_dic['Accuracy'].append(np.mean(acc_score))
#statallfeatures_dic['Standard Error'].append(np.std(acc_score))
#statallfeatures_dic['Sensitivity'].append(sensitivity)
#statallfeatures_dic['Specificity'].append(specificity)
#statallfeatures_dic['Precision'].append(precision)
#statallfeatures_dic['F1_Score'].append(f1_score)

#indaccs_dic['LogReg']=acc_score

---Run time is 289.9464748 seconds ---

Accuracy of each fold: 
 [0.8298368298368298, 0.8387850467289719, 0.8411214953271028, 0.8341121495327103, 0.8200934579439252]
Avg accuracy: 0.8327897958739079
Std of accuracy : 
0.007443465228519795

[[ 499  199]
 [ 159 1284]]

              precision    recall  f1-score   support

           0       0.76      0.71      0.74       698
           1       0.87      0.89      0.88      1443

    accuracy                           0.83      2141
   macro avg       0.81      0.80      0.81      2141
weighted avg       0.83      0.83      0.83      2141

Sensitivity: 0.8898128898128899
Specificity: 0.7148997134670487
precision: 0.7583586626139818
f1_score: 0.7359882005899705


In [18]:
model =  GradientBoostingClassifier(n_estimators = 500, 
                                           learning_rate = 0.1, 
                                           max_depth = 4, 
                                           random_state = 2)
acc_score = [];
Truth = [];
Output = [];
testing = df2021.iloc[:,-1]
model.fit(df2017.iloc[:, 5:18], df2017.iloc[:,-1])
pred_values = model.predict(df2021.iloc[:, 5:18])

acc = accuracy_score(testing, pred_values)
acc_score.append(acc)
Truth.extend(testing.values.reshape(testing.shape[0])) 
Output.extend(pred_values)

cm = confusion_matrix(Truth, Output)

sensitivity = cm[0][0]/(cm[0][0]+cm[0][1])
specificity = cm[1][1]/(cm[1][0]+cm[1][1])
precision = (cm[0][0])/(cm[0][0]+cm[1][0])
f1_score = (2*precision*sensitivity)/(precision+sensitivity)


print("Sensitivity: {}".format(specificity))
print("Specificity: {}".format(sensitivity))
print("precision: {}".format(precision))
print("f1_score: {}".format(f1_score))
print("Accuracy: {}".format(acc_score))


Sensitivity: 0.9315846994535519
Specificity: 0.257847533632287
precision: 0.6475225225225225
f1_score: 0.36882617062219375
Accuracy: [0.710800881704629]


In [19]:
random.seed(1000)
bigtest_df = dffull2021
finalPredicted = model.predict(bigtest_df.iloc[:,5:18])
bigtest_df["Veg_class"] = finalPredicted

In [20]:
woodyarea = 0
nonwoodyarea = 0 
totalarea = 0
index = -1
woody = 0
nw = 0
#print(bigtest_df.iat[index,4])
area = bigtest_df["Shape_Area"]
#print(area)

for i in bigtest_df["Veg_class"]:
    index += 1
    if i == 1:
        totalarea += bigtest_df.iat[index,4]
        woodyarea += bigtest_df.iat[index,4]
        #print(woodyarea)
        woody += 1
    if i == 0:
        totalarea += bigtest_df.iat[index,4]
        nonwoodyarea += bigtest_df.iat[index,4]
        #print(nonwoodyarea)
        nw += 1

FWCgrad = woodyarea / totalarea * 100

In [21]:
print(FWCada)
print(FWCdt)
print(FWCbag)
print(FWCgrad)

89.87153289926091
87.38374729095261
86.7726733842285
80.42383647059538
